In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui
import screen_brightness_control as sbc
from ctypes import cast, POINTER

from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize MediaPipe and system audio
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Webcam
cap = cv2.VideoCapture(0)
cv2.namedWindow("Webcam", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Webcam", 1280, 720)

zoom_factor = 1.0
base_zoom_distance = None

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm = hand_landmarks.landmark

            # Coordinates
            thumb_tip = np.array([lm[mp_hands.HandLandmark.THUMB_TIP].x * w, lm[mp_hands.HandLandmark.THUMB_TIP].y * h])
            index_tip = np.array([lm[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * w, lm[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * h])
            middle_tip = np.array([lm[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * w, lm[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * h])

            # Volume/Brightness Control
            vol_dist = np.linalg.norm(thumb_tip - index_tip)
            volume_level = np.clip(vol_dist / 200, 0.0, 1.0)
            brightness_level = np.clip(int(vol_dist / 2), 0, 100)

            try:
                volume.SetMasterVolumeLevelScalar(volume_level, None)
                sbc.set_brightness(brightness_level)
            except Exception as e:
                print("Error setting volume/brightness:", e)

            # Zoom Control
            zoom_dist = np.linalg.norm(index_tip - middle_tip)
            if base_zoom_distance is None:
                base_zoom_distance = zoom_dist
            zoom_factor = np.clip(1 + (zoom_dist - base_zoom_distance) / 100, 1.0, 2.0)

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Apply zoom
    try:
        center_x, center_y = int(w / 2), int(h / 2)
        radius_x, radius_y = int(w / (2 * zoom_factor)), int(h / (2 * zoom_factor))
        min_x, max_x = max(center_x - radius_x, 0), min(center_x + radius_x, w)
        min_y, max_y = max(center_y - radius_y, 0), min(center_y + radius_y, h)

        cropped = frame[min_y:max_y, min_x:max_x]
        zoomed = cv2.resize(cropped, (w, h))
    except:
        zoomed = frame  # Fallback

    cv2.imshow("Webcam", zoomed)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
